In [37]:
import pandas as pd
import numpy as np
import re
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import bisect
from tqdm import tqdm
import bottleneck as bn
from typing import Tuple

In [38]:
# 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
import pandas as pd
import re

# 학습 데이터와 메타데이터 로드 
train_ratings = pd.read_csv('/data/ephemeral/home/data/train/train_ratings.csv')
titles = pd.read_csv('/data/ephemeral/home/data/train/titles.tsv', sep='\t')
years = pd.read_csv('/data/ephemeral/home/data/train/years.tsv', sep='\t')

# 누락된 item 확인
train_items = set(train_ratings['item'].unique())
years_items = set(years['item'].unique())

missing_years = train_items - years_items

# 누락된 item의 제목 확인
missing_years_titles = titles[titles['item'].isin(missing_years)].copy()

# 연도 추출을 위한 함수 정의
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return int(match.group(1))
    else:
        return -1  # 연도를 찾지 못한 경우 -1로 설정

# 누락된 item의 연도 추출
missing_years_titles['year'] = missing_years_titles['title'].apply(extract_year)

# 추출된 연도 정보를 years 데이터프레임에 추가
years = pd.concat([years, missing_years_titles[['item', 'year']]], ignore_index=True)

# 학습 데이터와 업데이트된 years 데이터 병합
train_ratings = train_ratings.merge(years, on='item', how='left')

In [40]:
def reindex_column(data, column_name,start_index=1):
    """
    Reindex a column in the dataframe to ensure continuous indices starting from 0.

    Parameters:
    - data: pd.DataFrame, the input dataframe.
    - column_name: str, the column to reindex.

    Returns:
    - data: pd.DataFrame, the dataframe with reindexed column.
    - mapping_dict: dict, the original-to-new mapping dictionary.
    """
    unique_values = data[column_name].unique()
    mapping_dict = {original_id: new_id for new_id, original_id in enumerate(unique_values, start=start_index)}
    data[column_name] = data[column_name].map(mapping_dict)
    return data, mapping_dict

# 유저 및 아이템 리인덱싱
train_ratings, usr2idx_dict = reindex_column(train_ratings, 'user', start_index=0)  # Users can start from 0
train_ratings, item2idx_dict = reindex_column(train_ratings, 'item', start_index=1)  # Items start from 1


In [41]:
# train_validation_split 함수 정의
def train_validation_split(interaction_matrix: csr_matrix, num_random_items: int = 2) -> Tuple[csr_matrix, csr_matrix]:
    """
    Split a CSR interaction matrix into training and validation sets with specific rules:
    - Last interaction is always included in the validation set.
    - Additional 'num_random_items' interactions are randomly selected for the validation set.

    Parameters:
    - interaction_matrix: csr_matrix, the full user-item interaction matrix.
    - num_random_items: int, number of additional random interactions to include in the validation set.

    Returns:
    - train_matrix: csr_matrix, training set interactions.
    - validation_matrix: csr_matrix, validation set interactions.
    """
    train_rows, train_cols, train_data = [], [], []
    val_rows, val_cols, val_data = [], [], []

    num_users = interaction_matrix.shape[0]

    for user in range(num_users):
        # Get the non-zero interactions (item indices) for this user
        item_indices = interaction_matrix[user].indices
        num_items = len(item_indices)

        if num_items == 0:
            continue  # Skip users with no interactions

        # Last interaction is always included in the validation set
        val_items = [item_indices[-1]]

        # Determine the number of random items to sample
        if num_items > 1:
            available_items = item_indices[:-1]
            actual_num_random = min(num_random_items, len(available_items))
            if actual_num_random > 0:
                random_items = np.random.choice(available_items, size=actual_num_random, replace=False)
                val_items.extend(random_items)

        # Add the remaining items to the training set
        train_items = list(set(item_indices) - set(val_items))

        # Add training interactions
        train_rows.extend([user] * len(train_items))
        train_cols.extend(train_items)
        train_data.extend([1] * len(train_items))

        # Add validation interactions
        val_rows.extend([user] * len(val_items))
        val_cols.extend(val_items)
        val_data.extend([1] * len(val_items))

    # Create CSR matrices for training and validation
    train_matrix = csr_matrix((train_data, (train_rows, train_cols)), dtype='float64', shape=interaction_matrix.shape)
    validation_matrix = csr_matrix((val_data, (val_rows, val_cols)), dtype='float64', shape=interaction_matrix.shape)

    return train_matrix, validation_matrix

# 사용자-아이템 인터랙션 행렬 생성
rows, cols = train_ratings['user'].values, train_ratings['item'].values
feedback = np.ones_like(rows)

num_users, num_items = train_ratings['user'].nunique(), train_ratings['item'].nunique()
interaction_matrix = csr_matrix((feedback, (rows, cols)), dtype='float64', shape=(num_users, num_items + 1))

# Train/Validation 분할 (num_random_items를 2로 설정)
train_matrix, validation_matrix = train_validation_split(interaction_matrix, num_random_items=2)


In [42]:
def Recall_at_k_batch(X_pred, heldout_batch, k=10):
    """
    Recall@k를 계산합니다.
    
    Parameters:
    - X_pred: numpy.ndarray, 모든 아이템에 대한 예측 점수.
    - heldout_batch: numpy.ndarray or csr_matrix, 실제 상호작용 행렬.
    - k: int, 컷오프 값.
    
    Returns:
    - recall: numpy.ndarray, 각 사용자에 대한 Recall@k.
    """
    batch_users = X_pred.shape[0]
    idx = bn.argpartition(-X_pred, k, axis=1)[:, :k]  # 상위 k 인덱스 찾기
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx] = True

    # heldout_batch가 희소 행렬인 경우 밀집 배열로 변환
    if isinstance(heldout_batch, np.ndarray):
        X_true_binary = heldout_batch > 0
    else:
        X_true_binary = (heldout_batch > 0).toarray()

    # Recall@k 계산
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=10):
    """
    NDCG@k를 계산합니다.
    
    Parameters:
    - X_pred: numpy.ndarray, 모든 아이템에 대한 예측 점수.
    - heldout_batch: numpy.ndarray or csr_matrix, 실제 상호작용 행렬.
    - k: int, 컷오프 값.
    
    Returns:
    - ndcg: numpy.ndarray, 각 사용자에 대한 NDCG@k.
    """
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)[:, :k]
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk_part]
    idx_part = np.argsort(-topk_part, axis=1)
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]

    # heldout_batch가 희소 행렬인 경우 밀집 배열로 변환
    if isinstance(heldout_batch, np.ndarray):
        X_true_binary = heldout_batch > 0
    else:
        X_true_binary = (heldout_batch > 0).toarray()

    # DCG 계산
    tp = 1. / np.log2(np.arange(2, k + 2))
    DCG = (X_true_binary[np.arange(batch_users)[:, np.newaxis], idx_topk] * tp).sum(axis=1)

    # IDCG 계산
    IDCG = np.array([tp[:min(n, k)].sum() for n in X_true_binary.sum(axis=1)])
    ndcg = DCG / IDCG
    ndcg[np.isnan(ndcg)] = 0.0  # 상호작용이 없는 사용자에 대한 NaN 처리
    return ndcg

In [43]:
def crop(sequence, max_length=50, padding_value=0):
    """
    Crop the sequence to a maximum length from the end and pad if necessary.
    
    Parameters:
    - sequence: list of int, 아이템 인덱스 시퀀스.
    - max_length: int, 최대 시퀀스 길이.
    - padding_value: int, 패딩에 사용할 값 (기본값: 0).
    
    Returns:
    - sequence: list of int, 고정된 길이의 시퀀스.
    """
    if len(sequence) > max_length:
        sequence = sequence[-max_length:]
    else:
        sequence = [padding_value] * (max_length - len(sequence)) + sequence
    return sequence

def mask(sequence, mask_ratio=0.2, mask_token=0):
    """
    Mask a ratio of items in the sequence by replacing them with a special token (e.g., 0).
    
    Parameters:
    - sequence: list of int, 아이템 인덱스 시퀀스.
    - mask_ratio: float, 마스킹 비율 (기본값: 0.2).
    - mask_token: int, 마스킹에 사용할 토큰 (기본값: 0).
    
    Returns:
    - sequence: list of int, 마스킹된 시퀀스.
    """
    sequence = sequence.copy()
    num_to_mask = max(1, int(len(sequence) * mask_ratio))
    mask_indices = np.random.choice(len(sequence), num_to_mask, replace=False)
    for idx in mask_indices:
        sequence[idx] = mask_token  # Assuming 0 is the padding/masking index
    return sequence

def reorder(sequence, reorder_ratio=0.2):
    """
    Reorder a ratio of items in the sequence by shuffling them.
    
    Parameters:
    - sequence: list of int, 아이템 인덱스 시퀀스.
    - reorder_ratio: float, 재배열 비율 (기본값: 0.2).
    
    Returns:
    - sequence: list of int, 재배열된 시퀀스.
    """
    sequence = sequence.copy()
    num_to_reorder = max(1, int(len(sequence) * reorder_ratio))
    reorder_indices = np.random.choice(len(sequence), num_to_reorder, replace=False)
    reordered_items = [sequence[idx] for idx in reorder_indices]
    np.random.shuffle(reordered_items)
    for i, idx in enumerate(reorder_indices):
        sequence[idx] = reordered_items[i]
    return sequence

In [44]:
class SequentialDataset(Dataset):
    def __init__(self, train_matrix: csr_matrix, validation_matrix: csr_matrix, max_seq_length: int = 50, augmentations: list = None):
        """
        사용자-아이템 상호작용 행렬을 기반으로 시퀀스를 생성하고, 데이터 증강을 적용합니다.

        Parameters:
        - train_matrix: csr_matrix, 훈련 데이터 상호작용 행렬.
        - validation_matrix: csr_matrix, 검증 데이터 상호작용 행렬.
        - max_seq_length: int, 최대 시퀀스 길이.
        - augmentations: list of functions, 데이터 증강 함수 리스트.
        """
        self.train_matrix = train_matrix
        self.validation_matrix = validation_matrix
        self.num_users, self.num_items = train_matrix.shape
        self.max_seq_length = max_seq_length
        self.augmentations = augmentations if augmentations is not None else []

        # Preprocess: create a list of (user, target) pairs
        self.samples = []
        for user in range(self.num_users):
            train_item_indices = set(train_matrix[user].indices)
            val_item_indices = validation_matrix[user].indices
            if len(val_item_indices) == 0:
                continue  # Skip users with no validation items
            for target in val_item_indices:
                # Define the training sequence as all train items for the user
                sequence = list(train_item_indices)
                self.samples.append((user, sequence, target))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        user, sequence, target = self.samples[idx]

        # Pad or truncate the sequence to max_seq_length
        if len(sequence) > self.max_seq_length:
            sequence_padded = sequence[-self.max_seq_length:]
        else:
            sequence_padded = [0] * (self.max_seq_length - len(sequence)) + sequence  # Assuming 0 is the padding index

        # Apply augmentations to create two views
        if len(self.augmentations) >= 2:
            a1, a2 = random.sample(self.augmentations, 2)
            augmented_seq1 = a1(sequence_padded)
            augmented_seq2 = a2(sequence_padded)
        elif len(self.augmentations) == 1:
            a1 = self.augmentations[0]
            augmented_seq1 = a1(sequence_padded)
            augmented_seq2 = a1(sequence_padded)
        else:
            augmented_seq1 = sequence_padded
            augmented_seq2 = sequence_padded

        # Ensure augmented sequences are of fixed length
        if len(augmented_seq1) > self.max_seq_length:
            augmented_seq1 = augmented_seq1[-self.max_seq_length:]
        elif len(augmented_seq1) < self.max_seq_length:
            augmented_seq1 = [0] * (self.max_seq_length - len(augmented_seq1)) + augmented_seq1

        if len(augmented_seq2) > self.max_seq_length:
            augmented_seq2 = augmented_seq2[-self.max_seq_length:]
        elif len(augmented_seq2) < self.max_seq_length:
            augmented_seq2 = [0] * (self.max_seq_length - len(augmented_seq2)) + augmented_seq2

        return {
            'seq1': torch.tensor(augmented_seq1, dtype=torch.long),
            'seq2': torch.tensor(augmented_seq2, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long)
        }

In [45]:
class CL4SRecModel(nn.Module):
    def __init__(self, num_items, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1):
        """
        CL4SRec 모델 정의.
        
        Parameters:
        - num_items: int, 아이템의 총 개수.
        - embed_dim: int, 임베딩 차원.
        - num_heads: int, Transformer의 헤드 수.
        - num_layers: int, Transformer 인코더 레이어 수.
        - dropout: float, 드롭아웃 비율.
        """
        super(CL4SRecModel, self).__init__()
        self.num_items = num_items
        self.embed_dim = embed_dim

        # Item embedding (0은 패딩 인덱스)
        self.item_embedding = nn.Embedding(num_items + 1, embed_dim, padding_idx=0)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Prediction layer for next item prediction
        self.predictor = nn.Linear(embed_dim, num_items +1)

    def forward(self, sequences):
        """
        순전파 함수.
        
        Parameters:
        - sequences: tensor of shape (batch_size, seq_length)
        
        Returns:
        - logits: tensor of shape (batch_size, num_items +1)
        - encoded: tensor of shape (batch_size, embed_dim)
        """
        # Embed the sequences
        embedded = self.item_embedding(sequences)  # (batch_size, seq_length, embed_dim)

        # Transformer expects input of shape (seq_length, batch_size, embed_dim)
        embedded = embedded.transpose(0, 1)

        # Pass through Transformer encoder
        encoded = self.transformer_encoder(embedded)  # (seq_length, batch_size, embed_dim)

        # Take the representation of the last time step
        encoded = encoded[-1, :, :]  # (batch_size, embed_dim)

        # Predict the next item
        logits = self.predictor(encoded)  # (batch_size, num_items +1)

        return logits, encoded  # Return both logits and encoded representations

In [46]:
def contrastive_loss(encoded1, encoded2, temperature=0.07):
    """
    두 인코딩된 시퀀스 간의 대조 손실을 계산합니다.
    
    Parameters:
    - encoded1: tensor of shape (batch_size, embed_dim)
    - encoded2: tensor of shape (batch_size, embed_dim)
    - temperature: float, 스케일링을 위한 온도 파라미터
    
    Returns:
    - loss: tensor, 대조 손실 값
    """
    batch_size = encoded1.size(0)
    
    # 임베딩 정규화
    encoded1 = nn.functional.normalize(encoded1, dim=1)
    encoded2 = nn.functional.normalize(encoded2, dim=1)

    # 유사도 매트릭스 계산
    logits = torch.matmul(encoded1, encoded2.T) / temperature  # (batch_size, batch_size)

    # 정답 레이블 생성 (대각선 위치가 정답)
    labels = torch.arange(batch_size).to(encoded1.device)

    # 교차 엔트로피 손실 계산
    loss = nn.CrossEntropyLoss()(logits, labels)

    return loss

In [47]:
# 데이터 증강 리스트
augmentations = [crop, mask, reorder]

# Dataset 및 DataLoader 생성
train_dataset = SequentialDataset(train_matrix, validation_matrix, max_seq_length=50, augmentations=augmentations)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)

# 데이터셋 일부를 확인하여 시퀀스 길이가 올바른지 확인
for i in range(5):
    sample = train_dataset[i]
    print(f"Sample {i}:")
    print(f"Seq1 length: {len(sample['seq1'])}, Seq2 length: {len(sample['seq2'])}, Target: {sample['target']}")
    print(f"Seq1: {sample['seq1']}")
    print(f"Seq2: {sample['seq2']}")
    print()

Sample 0:
Seq1 length: 50, Seq2 length: 50, Target: 34
Seq1: tensor([326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339,
        340, 341, 342, 343, 370, 361, 346, 347, 348, 371, 350, 351, 352, 349,
        354, 355, 356, 359, 358, 345, 360, 344, 362, 363, 364, 365, 366, 367,
        368, 369, 357, 353, 372, 373, 374, 375])
Seq2: tensor([326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339,
        340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353,
        354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367,
        368, 369, 370, 371, 372, 373, 374, 375])

Sample 1:
Seq1 length: 50, Seq2 length: 50, Target: 168
Seq1: tensor([326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339,
        340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353,
        354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367,
        368, 369, 370, 371, 372, 373, 374, 375])
Seq2: tensor([  0,   

In [48]:
def train(model, train_loader, optimizer, epoch, temperature=0.07):
    """
    모델을 학습하는 함수.
    
    Parameters:
    - model: nn.Module, 학습할 모델.
    - train_loader: DataLoader, 학습 데이터로더.
    - optimizer: torch.optim.Optimizer, 옵티마이저.
    - epoch: int, 현재 에폭 번호.
    - temperature: float, 대조 손실 계산을 위한 온도 파라미터.
    """
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        seq1 = batch['seq1'].to(device)  # (batch_size, seq_length)
        seq2 = batch['seq2'].to(device)  # (batch_size, seq_length)
        targets = batch['target'].to(device)  # (batch_size)

        optimizer.zero_grad()

        # seq1에 대한 순전파
        logits1, encoded1 = model(seq1)  # logits1: (batch_size, num_items +1), encoded1: (batch_size, embed_dim)

        # seq2에 대한 순전파
        logits2, encoded2 = model(seq2)  # logits2: (batch_size, num_items +1), encoded2: (batch_size, embed_dim)

        # 다음 아이템 예측 손실
        loss_pred1 = nn.functional.cross_entropy(logits1, targets)
        loss_pred2 = nn.functional.cross_entropy(logits2, targets)
        loss_pred = (loss_pred1 + loss_pred2) / 2

        # 대조 손실
        loss_contrast = (contrastive_loss(encoded1, encoded2, temperature) + 
                         contrastive_loss(encoded2, encoded1, temperature)) / 2

        # 총 손실
        loss = loss_pred + loss_contrast

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")

In [49]:
def evaluate(model, validation_matrix, train_matrix, k=10):
    """
    모델을 평가하는 함수.
    
    Parameters:
    - model: nn.Module, 학습된 모델.
    - validation_matrix: csr_matrix, 검증 데이터 상호작용 행렬.
    - train_matrix: csr_matrix, 훈련 데이터 상호작용 행렬.
    - k: int, 컷오프 값.
    
    Returns:
    - mean_recall: float, 평균 Recall@k.
    - mean_ndcg: float, 평균 NDCG@k.
    - mean_loss: float, 평균 Validation Loss.
    """
    model.eval()
    loss_fn = nn.CrossEntropyLoss()
    total_loss = 0.0
    total_samples = 0
    all_recall = []
    all_ndcg = []
    batch_size = 256
    num_users = validation_matrix.shape[0]
    
    with torch.no_grad():
        for start in tqdm(range(0, num_users, batch_size), desc="Evaluating"):
            end = min(start + batch_size, num_users)
            users = np.arange(start, end)
            
            # 시퀀스 및 타겟 준비
            sequences = []
            val_targets = []
            for user in users:
                train_items = train_matrix[user].indices
                if len(train_items) > 0:
                    seq = train_items[-50:].tolist()
                else:
                    seq = []
                if len(seq) < 50:
                    seq = [0] * (50 - len(seq)) + seq  # 패딩
                sequences.append(seq)
                
                # 각 사용자에 대해 하나의 타겟만 선택 (예: 마지막 아이템)
                val_items = validation_matrix[user].indices
                if len(val_items) > 0:
                    val_targets.append(val_items[-1])  # 마지막 아이템을 타겟으로 사용
                else:
                    val_targets.append(0)  # 타겟이 없는 경우 0으로 패딩 (패딩 인덱스와 일치해야 함)
            
            if len(sequences) == 0:
                continue  # 검증 세트에 아이템이 없는 사용자 스킵
            
            sequences = torch.tensor(sequences, dtype=torch.long).to(device)  # (batch_size, seq_length)
            targets_tensor = torch.tensor(val_targets, dtype=torch.long).to(device)  # (batch_size)
            
            # 모델 예측
            logits, _ = model(sequences)  # logits: (batch_size, num_items +1)
            
            # 손실 계산
            loss_pred = loss_fn(logits, targets_tensor)
            total_loss += loss_pred.item() * len(users)
            total_samples += len(users)
            
            # 예측 점수 추출
            X_pred = logits.cpu().numpy()
            
            # 실제 타겟 이진 행렬 생성 (Recall@k, NDCG@k 계산을 위해)
            heldout_batch = validation_matrix[users].toarray()
            
            # Recall@k 및 NDCG@k 계산
            recall = Recall_at_k_batch(X_pred, heldout_batch, k)
            ndcg = NDCG_binary_at_k_batch(X_pred, heldout_batch, k)
            all_recall.extend(recall)
            all_ndcg.extend(ndcg)
    
    mean_recall = np.mean(all_recall) if all_recall else 0.0
    mean_ndcg = np.mean(all_ndcg) if all_ndcg else 0.0
    mean_loss = total_loss / total_samples if total_samples > 0 else 0.0
    
    print(f"Validation Loss: {mean_loss:.4f}, Recall@{k}: {mean_recall:.4f}, NDCG@{k}: {mean_ndcg:.4f}")
    return mean_recall, mean_ndcg, mean_loss

In [ ]:
# 모델 초기화
model = CL4SRecModel(num_items=num_items, embed_dim=256, num_heads=8, num_layers=4, dropout=0.1)

model = model.to(device)

from torch.optim.lr_scheduler import ReduceLROnPlateau

# 옵티마이저 및 스케줄러 설정
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Early Stopping을 위한 변수 초기화
best_loss = float('inf')
patience = 7  # patience 값을 늘려 조기 중단을 지연시킴
counter = 0

# 모델 학습 및 평가 루프
num_epochs = 60
for epoch in range(1, num_epochs + 1):
    train(model, train_loader, optimizer, epoch)
    recall, ndcg, val_loss = evaluate(model, validation_matrix, train_matrix, k=10)
    # Early Stopping 체크
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        # 최적 모델 저장
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

Epoch 1:   0%|          | 0/368 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 368/368 [00:39<00:00,  9.43it/s]


Epoch 1, Loss: 9.0659


Evaluating: 100%|██████████| 123/123 [00:13<00:00,  9.36it/s]


Validation Loss: 8.4970, Recall@10: 0.0257, NDCG@10: 0.0171


...

Validation Loss: 6.4522, Recall@10: 0.2247, NDCG@10: 0.1834


Epoch 60: 100%|██████████| 368/368 [00:39<00:00,  9.37it/s]


Epoch 60, Loss: 2.6307


Evaluating: 100%|██████████| 123/123 [00:13<00:00,  8.88it/s]


Validation Loss: 6.3972, Recall@10: 0.2290, NDCG@10: 0.1873


# 제출 파일 생성하기

In [60]:
# 사용자 및 아이템 ID 매핑 딕셔너리 생성
idx2usr_dict = {v: k for k, v in usr2idx_dict.items()}
idx2item_dict = {v: k for k, v in item2idx_dict.items()}

In [63]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

def generate_recommendations(model, train_matrix, idx2usr_dict, idx2item_dict, top_k=10, device='cpu'):
    """
    Generate top-k recommendations for each user after masking existing interactions and excluding padding.

    Parameters:
    - model: Trained PyTorch model.
    - train_matrix: csr_matrix, training data interaction matrix.
    - idx2usr_dict: dict, mapping from internal user indices to original user IDs.
    - idx2item_dict: dict, mapping from internal item indices to original item IDs.
    - top_k: int, number of recommendations to generate per user.
    - device: str, device to perform computations ('cpu' or 'cuda').

    Returns:
    - recommendations_df: pd.DataFrame, user-to-top-k recommended items with one item per row.
    """
    model.eval()  # 평가 모드로 전환
    num_users = train_matrix.shape[0]
    num_items = train_matrix.shape[1]
    
    recommendations = []

    with torch.no_grad():
        for user_idx in tqdm(range(num_users), desc="Generating Recommendations"):
            # 사용자의 상호작용 아이템 가져오기
            train_items = set(train_matrix[user_idx].indices)
            
            # 시퀀스 준비: 마지막 50개 아이템, 패딩은 0
            seq = list(train_items)[-50:] if len(train_items) >= 50 else list(train_items)
            if len(seq) < 50:
                seq = [0] * (50 - len(seq)) + seq  # 패딩
            elif len(seq) > 50:
                seq = seq[-50:]
            
            # 시퀀스를 텐서로 변환하고 디바이스로 이동
            seq_tensor = torch.tensor([seq], dtype=torch.long).to(device)  # Shape: (1, 50)
            
            # 모델을 통해 예측 점수 계산
            logits, _ = model(seq_tensor)  # logits: (1, num_items +1)
            scores = logits.cpu().numpy()[0]  # Shape: (num_items +1,)
            
            # 마스킹: 이미 상호작용한 아이템과 패딩 인덱스 `0` 제외
            scores[list(train_items)] = -np.inf
            scores[0] = -np.inf  # 패딩 인덱스 제외
            
            # 상위 k개 아이템 선택
            top_items = np.argsort(-scores)[:top_k]
            
            # 추천 아이템 매핑 및 필터링
            recommended_items = [idx2item_dict.get(item_idx, 0) for item_idx in top_items 
                                 if item_idx != 0 and item_idx not in train_items]
            
            # 추천 아이템이 부족할 경우, 랜덤 아이템 추가
            if len(recommended_items) < top_k:
                available_items = set(range(1, num_items +1)) - train_items - set(top_items)
                if available_items:
                    remaining = top_k - len(recommended_items)
                    additional_items = np.random.choice(list(available_items), size=remaining, replace=False)
                    recommended_items += [idx2item_dict.get(item, 0) for item in additional_items]
            
            # 추천 아이템이 정확히 top_k개인지 확인
            recommended_items = recommended_items[:top_k]
            while len(recommended_items) < top_k:
                # 추가 아이템이 없을 경우, 다른 전략을 적용할 수 있습니다.
                # 여기서는 임의로 0을 추가하지만, 실제로는 다른 아이템으로 대체하는 것이 좋습니다.
                recommended_items.append(0)
            
            # 사용자 ID 매핑
            original_user_id = idx2usr_dict.get(user_idx, user_idx)
            
            # 추천 아이템을 개별 행으로 추가
            for item in recommended_items:
                if item != 0:  # 0이 아닌 경우에만 추가
                    recommendations.append({'user': original_user_id, 'item': item})
    
    # DataFrame 생성
    recommendations_df = pd.DataFrame(recommendations)
    
    # CSV 파일로 저장
    recommendations_df.to_csv('submission.csv', index=False)
    print("Submission file saved as 'submission.csv'")
    
    return recommendations_df

In [64]:
# 최적 모델 로드
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)
model.eval()

# 추천 생성 함수 호출
recommendations_df = generate_recommendations(model, train_matrix, idx2usr_dict, idx2item_dict, top_k=10, device=device)

# 추천 결과 확인
print(recommendations_df.head())

Generating Recommendations:   0%|          | 0/31360 [00:00<?, ?it/s]

Generating Recommendations: 100%|██████████| 31360/31360 [01:39<00:00, 314.00it/s]


Submission file saved as 'submission.csv'
   user   item
0    11    173
1    11     19
2    11   4226
3    11  68941
4    11   8830
